<center>
<table>
  <tr>
    <td><img src="http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://github.com/astg606/py_materials/blob/master/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>

---

<center><h1> <font color="red">Reading LANDSAT hdf Files using pyhdf</font></h1></center>

This Jupyter notebook shows an example of how to use the **pyhdf**, **Numpy**, **Xarray**, **Matplotlib**, **Cartopy**, and **hvplot** Python packages to work with a LANDSAT file in HDF4 format.  

The main workflow steps are:
- Open a LANDSAT HDF4 file
- Read the global file metadata
    - Recognize the file attribute
    - Find names of variables and their attributes
- Read dataset from file
- Visualize satellite data on a map

## <font color="red">Primary References/Resources</font>

- https://moonbooks.org/Articles/How-to-read-a-MODIS-HDF-file-using-python-/
- http://fhs.github.io/pyhdf/modules/SD.html 

## <font color="red">Import the Python Packages</font>

Six Python packages (libraries)  used in this Notebook:
- **pyhdf**: Read HDF4 files
- **NumPy**: Perform array operations
- **Xarray**: Work with labeled multi-dimensional arrays
- **Matplotlib**: Make static plots (mainly two-dimensional)
- **Cartopy**: Create maps
- **hvplot**: Create interactive plots/maps

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pprint
import os

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import hvplot.xarray
from cartopy import crs as ccrs

In [ ]:
from pyhdf.SD import SD
from pyhdf.SD import SDS
from pyhdf.SD import SDC
from pyhdf.SD import SDim
from pyhdf.SD import SDAttr

In [ ]:
print(f"Version of Numpy:   {np.__version__}")
print(f"Version of Xarray:  {xr.__version__}")

## <font color="red">LANDSAT Satellite Program</font>

[The Landsat Program](https://www.usgs.gov/landsat-missions/landsat-satellite-missions) is a series of Earth-observing satellite missions jointly managed by NASA and the U.S. Geological Survey. Landsat satellites have the optimal ground resolution and spectral bands to efficiently track land use and to document land change due to climate change, urbanization, drought, wildfire, biomass changes (carbon assessments), and a host of other natural and human-caused changes. 

The Landsat Program’s continuous archive (1972-present) provides essential land change data and trending information not otherwise available. Landsat represents the world’s longest continuously acquired collection of space-based moderate-resolution land remote sensing data. Landsat is an essential capability that enables the U.S. Department of the Interior to wisely manage Federal lands. People around the world are using Landsat data for research, business, education, and other activities. 

#### LANDSAT Data File Name Convention

Details on the naming convention can be found in:

https://gisgeography.com/landsat-file-naming-convention/
    


## <font color="red"> Accessing a Sample HDF4 Data Files</font>

### <font color="blue"> Step 1: Identify the Location of the File</font>

Directory where the LANDSAT file is located:

In [ ]:
#data_dir = "/Users/jkouatch/myTasks/PythonTraining/ASTG606/Materials/sat_data/LANDSAT_Data/"
data_dir = "/tljh-data/sat_data/LANDSAT_Data"

Full path to the file name:

In [ ]:
fname = os.path.join(data_dir, "LT50830152011198GLC00.hdf")

Decoding the naming convention on `fname`:
- `L`: Landsat
- `T`: IRS sensor
- `5`: Landsat-5
- `083`: Worldwide Reference System (WRS) path
- `015`: WRS row (the north-south row that sectionalizes WRS)
- `2011`: year
- `214`: Julian day of the year
- `GLC`: ground station identifier
- `00`: archive version number


### <font color="blue"> Step 2: Open the File</font>

Opening files for reading:

In [ ]:
fid = SD(fname, SDC.READ)

### <font color="blue"> Step 3: Obtain the File Attributes</font>

Basic information on the files:

- First number: number of datasets in the file (not to be confused with Xarray datasets)
- Second number: number of attributes attached to the global file.

In [ ]:
fid.info()

#### File Attributes

- We can access the file attributes which hold important global metadata.
- Some notable ones are the data provider, satellite name and instrument, coordinate boundaries, and structural metadata.

In [ ]:
file_attrs = fid.attributes()
for index, name in enumerate(file_attrs.keys(), start=1):
    print(f"{index:>2}- {name}: --> {file_attrs[name]}")

#pprint.pprint(file_attrs)

We can also access the datasets' names and basic info such as shape and dimension labels

In [ ]:
file_dts = fid.datasets()
for index, name in enumerate(file_dts.keys(), start=1):
    print(f"{index:>2}- {name} \n\t {file_dts[name]}")

### <font color="blue">Step 4: Data Extraction</font>

Let's assume that we want to extract data from the field `sr_band4`.

The `select()` method from the `SD` class allows us to extract a dataset (object) given it's name or index number.

In [ ]:
field_name = 'sr_band4'

In [ ]:
sample_ds = fid.select(field_name) 

Get basic information from the dataset:

- The `info()` function in the `SDS` class allows us to get the dataset name, rank (or level with file-leve being rank 1), dimension lengths, data type, and number of attributes.

In [ ]:
sample_ds.info()

List the dataset attributes:

In [ ]:
ds_attrs = sample_ds.attributes()
ds_attrs

- Important information such as `_FillValue`, `scale_factor`, and `add_offset` are found in dataset attributes and will be important to fully restoring the data.
- It is good practice to set default values if using different datasets/variables.

In [ ]:
_FillValue = None
scale_factor = 1
add_offset = 0

In [ ]:
for key, value in sample_ds.attributes().items():
    if key == '_FillValue':
        _FillValue = value  
    if key == 'scale_factor':
        scale_factor = value
    if key == 'add_offset':
        add_offset = value
    
print(f'Fill Value:   {_FillValue}')
print(f'Scale Factor: {scale_factor}')
print(f'Offset:       {add_offset}')

#### Extract the data

- We can retrieve and store the data itself as a NumPy array using the `get()` function.

In [ ]:
sample_data = sample_ds.get() 

Confirms that the data has been stored as a NumPy array.

In [ ]:
print("Dataset Class Type: ", type(sample_data))

In [ ]:
sample_data

Just like any NumPy array, we can get the shape and dtype.

In [ ]:
sample_data.shape

In [ ]:
sample_data.dtype

### <font color="blue">Step 5: Get the Dimensions</font>

- From the `SDS` class, we can access the dimension names and sizes using the `dimensions()` function.

In [ ]:
ds_dims = sample_ds.dimensions()
ds_dims

- While this is nice, the dictionary above does not allow us to access actual dimension objects and the other information that they may hold. 
- To access the objects, we can use the `dim()` function from the `SDS` class.

In [ ]:
sample_dims = list()   

for i in range(len(ds_dims)):
    sample_dims.append(sample_ds.dim(i))

sample_dims   

- We can access not only the labels and size but also the units, scale data type, and number of attributes. 
- We can also access the attributes.

In [ ]:
for i in range(len(sample_dims)):
    print(f"Dimension {i+1}")
    print("\tInfo:", sample_dims[i].info())
    print("\tLength:", sample_dims[i].length())
    print("\tAttributes: ", sample_dims[i].attributes())

### <font color="blue">Step 6: Coordinates</font>

- In `pyhdf`, it is possible that coordinates (known as dimension scales) are actually stored as datasets. 
- The `SDS` class provides the `iscoordvar()` function to determine that.

In [ ]:
print(bool(sample_ds.iscoordvar()))

# If there was a scale, it would be accessible via: dim1.getscale()

We can try to traverse through all the datasets and see if one of them holds coordinates:

In [ ]:
coord_sets = list() 

for i in range(len(fid.datasets())):
    ds = fid.select(i)
    if ds.iscoordvar():
        coord_sets.append(ds)

In [ ]:
coord_sets

- It is also possible that some coordinate information is stored as a file attribute. 
- If we go back once more to our global attribute dictionary, we can see some keys such as corner and bounding coordinates.

In [ ]:
pprint.pprint(fid.attributes())

We can then traverse through it and extract and attributes that may be related to the coordinates.

In [ ]:
coord_attrs = dict()   # Will hold coordinate-related attributes
for key, value in fid.attributes().items():
    if 'coordinate' in key.lower() or 'latlong' in key.lower():
        coord_attrs[key] = value

coord_attrs

Looking at these attributes, it would be much easier to work with our bounding coordinates.

If we now go back to our datasets, we can see that they all have the same shape:

In [ ]:
file_dts = fid.datasets()
for index, name in enumerate(file_dts.keys(), start=1):
    print(f"{index:>2}- {name} \t {file_dts[name][0:2]}")

In [ ]:
for idx, vals in enumerate(fid.datasets().values(), start=1):
    print(f"Dataset {idx:>3} Shape:", vals[0:2])
    shape = vals[1]

#vals

- Using the bounding coordinates and the shape, we can artificially create our full coordinates.
- However, this is under the assumption that the datasets truly align with this artificial system. This means that we can't confirm its accuracy.

The first step would be to extract our boundaries.

In [ ]:
for key in coord_attrs.keys():
    if 'north' in key.lower():
        latN = coord_attrs[key]
    if 'south' in key.lower():
        latS = coord_attrs[key]
    if 'east' in key.lower():
        lonE = coord_attrs[key]
    if 'west' in key.lower():
        lonW = coord_attrs[key]

print("North:", latN)
print("South:", latS)
print("East:", lonE)
print("West:", lonW)

Next, we would create our spacing using the dataset shapes and boundaries.

In [ ]:
# Creating our coords. 'lats' can be substituted for 'y' and 'lons' for 'x'
lat_space = (latN - latS) / shape[0]
lon_space = (lonE - lonW) / shape[1]

print(lat_space, "|", lon_space)

Now, we can finally create our coordinates!

In [ ]:
lats = np.linspace(latS, latN, shape[0])
lons = np.linspace(lonW, lonE, shape[1])

In [ ]:
print('Latitudes:\n', lats)
print('Longitudes:\n', lons)

We'll now close our file reader.

In [ ]:
fid.end()

## <font color="red"> Conversion to Xarray DataArrays and Datasets</font>


We will write a set of functions that we help us to:
- Read a LANDSAT data file
- Read a field and determine each dimension and coordinate.
- Read the field attribute and and restore its values.
- Create a Xarray DataArray associated with the field.

We can then combine all the DataArrays into a Xarray Dataset.

#### Function to open the file and get the file identifier

In [ ]:
def get_fid(file_name):
    """
    Open a HDF4 file and return thfile identifier.
    
    Parameters
    ----------
    file_name : str
         a file name

    Returns
    -------
    fid :
         an SD object (file identifier)
    """
    fid = SD(file_name, SDC.READ)
    return fid

#### Function to get the value of an attribute of a dataset

In [ ]:
def get_ds_attribute_value(sample_ds, attr_name):
    '''
    Obtain the value of a specified attribute in a dataset.
    
    Parameter
    ---------
    sample_ds : SDS object
    attr_name : str
         Attribute name    
    
    Returns
    --------
    value: float, int, str, list
         Value of the attribute. If attribute not available, None.
    '''
    for key, value in sample_ds.attributes().items():
        if key == attr_name:
            return value 
    return None

#### Function to restore the data from the dataset attributes

In [ ]:
def restore_data(sample_ds):
    '''
    Restore the data given the dataset attribute
    Parameter(s): SDS object
    Return Type(s): NumPy array
    Function: restores data of a given dataset object
    '''
    _FillValue = get_ds_attribute_value(sample_ds, '_FillValue')
    scale_factor = get_ds_attribute_value(sample_ds, 'scale_factor')
    add_offset = get_ds_attribute_value(sample_ds, 'add_offset')
    valid_range = get_ds_attribute_value(sample_ds, 'valid_range')
    
    data = sample_ds.get().astype('float')
    
    if valid_range:
        valid_min, valid_max = valid_range[0], valid_range[1]
        invalid = np.logical_or(data > valid_max, data < valid_min)
    
    #data[invalid] = np.nan
    #data = np.where(data != _FillValue, data, np.nan)
    
    if _FillValue:
        if valid_range:
            invalid = np.logical_or(invalid, data == _FillValue)
            data[invalid] = np.nan
        else:
            data = np.where(data != _FillValue, data, np.nan)
    
    if add_offset:
        data -= add_offset
    if scale_factor:
        data *= scale_factor
    data = np.ma.masked_array(data, np.isnan(data))
    return data

#### Function to obtain the dimension object from a dataset

In [ ]:
def get_dims(sample_ds):
    '''
    Get the dimension objects of a given dataset object
    
    Parameters
    ----------
    sample_ds : SDS object
    
    Returns
    -------
    sample_dim : list
          List of SDim objects
    '''
    sample_dims = list()
    
    for i in range(len(sample_ds.dimensions())):
        sample_dims.append(sample_ds.dim(i))
        
    return sample_dims

#### Function to obtain the dimension attributes from a dimension object

In [ ]:
def get_dim_attrs(dim):
    '''
    Get the attributes of a given dimension object.
    
    Parameters
    ----------
    dim : SDim object
    
    Returns
    -------
    attrs : dict
         Dictionary of attributes.
    '''
    attrs = dict()
    
    attrs['Given Name'] = dim.info()[0]
    attrs['dtype'] = dim.info()[2]
    attrs.update(dim.attributes())   
    
    return attrs

In [ ]:
def get_dims_attrs(ds):
    dims_attrs = dict()
    dims = get_dims(ds)
    
    for dim in dims:
        if dim.info()[0] == 'YDim':
            dims_attrs['lat'] = get_dim_attrs(dim)
        elif dim.info()[0] == 'XDim':
            dims_attrs['lon'] = get_dim_attrs(dim)
            
    return dims_attrs

#### Function to obtain the coordinates from the file identifier

In [ ]:

def get_fid_coords(sample_fid):
    '''
    Returns False if there are no file-level coords, otherwise True.
    
    Parameters
    ----------
    sample_fid : SD object
    
    Returns
    -------
    bool
      
    '''
    coord_sets = list()

    for i in range(len(sample_fid.datasets())):
        sample_ds = sample_fid.select(i)
        if bool(sample_ds.iscoordvar()):
            coord_sets.append(sample_ds)

    if coord_sets:
        return True
    else:
        return False

#### Function to obtain the coordinate bounds

In [ ]:
def get_coord_bounds(sample_fid):
    '''
    Obtain the latitude/longitude corners from file attributes.
    
    Parameters
    ----------
    sample_fid : SD object
    
    Returns
    coord_bounds : dict
          Dictionary of String keys and numeric items
    '''
    coord_attrs = {}    
    # Gets our coordinate-related attributes
    for key, value in sample_fid.attributes().items():
        if 'coordinate' in key.lower():
            coord_attrs[key] = value 
        
    coord_bounds = {}
    # Gets our coordinate bounds
    for key in coord_attrs.keys():
        if 'north' in key.lower():
            coord_bounds['latN'] = coord_attrs[key]
        if 'south' in key.lower():
            coord_bounds['latS'] = coord_attrs[key]
        if 'east' in key.lower():
            coord_bounds['lonE'] = coord_attrs[key]
        if 'west' in key.lower():
            coord_bounds['lonW'] = coord_attrs[key]    
    
    return coord_bounds

#### Function to compute the coordinate (latitude/longitude gridpoints)

In [ ]:
def get_ds_coords(sample_fid, sample_ds): 
    '''
    Determine the latitude/longitude grid points of a dataset.
 
    Parameters
    ----------
    sample_fid : SD object
    sample_ds : SDS object
    
    Returns
    -------
    sample_coords : dict
          Dictionary containing the latitude and longitudes coordinates.
    '''
    sample_bounds = get_coord_bounds(sample_fid)
    
    latN = sample_bounds['latN']
    latS = sample_bounds['latS']
    lonE = sample_bounds['lonE']
    lonW = sample_bounds['lonW']
    
    lat_shape = sample_ds.dimensions()['YDim']
    lon_shape = sample_ds.dimensions()['XDim']
    
    if isinstance(sample_bounds, dict):   # Have to configure dataset coords
        lat_space = (latN - latS) / lat_shape
        lon_space = (lonE - lonW) / lon_shape
        
        lats = np.linspace(latS, latN + lat_space, lat_shape)
        lons = np.linspace(lonW, lonE + lon_space, lon_shape)
        
        sample_coords = {'lats': lats, 'lons': lons}
        return sample_coords

#### Function to create a Xarray Dataset from a LANDSAT file

To simplify the file, we are not going to only consider the fields: `sr_band2`, `sr_band4`, and `toa_band6`.

In [ ]:
def create_xarray_dataset_from_file(file):
    '''
    Read a LANDSAT HDF4 file and returns an Xarray dataset.
    
    Parameters
    ----------
    file : str
         LANDSAT file name
    
    Returns
    -------
    xr_ds : Xarray Dataset
    '''
    
    list_attributes = ['long_name', 'units', 'app_version']
    sample_fields = ['sr_band2', 'sr_band4', 'toa_band6']
    
    fid = get_fid(file)
    
    if get_fid_coords(fid):   # File-level coords exist
        pass
    else:
        fid_coords = False    # File-level coords do not exist   
    
    xr_ds = xr.Dataset()
    
    for name in fid.datasets().keys():
        
        # We are only going to consider few fields
        if name not in sample_fields:
            continue
            
        ds = fid.select(name)
        
        coords_dict = get_ds_coords(fid, ds)
        lats = coords_dict['lats']
        lons = coords_dict['lons']
        
        xr_ds[name] = xr.DataArray(restore_data(ds), 
                                   coords = [lats, lons], 
                                   dims = ['lat', 'lon'])
        for key in list_attributes:
            xr_ds[name].attrs[key] = get_ds_attribute_value(ds, key)
        #xr_ds[name].attrs = ds.attributes()
        
        dims = get_dims(ds)
        for dim in dims:
            if dim.info()[0] == 'YDim':
                xr_ds[name].lat.attrs = get_dim_attrs(dim)
            elif dim.info()[0] == 'XDim':
                xr_ds[name].lon.attrs = get_dim_attrs(dim)
    
    xr_ds.attrs = fid.attributes()
    
    fid.end()
    return xr_ds

#### Create a Xarray Dataset from a LANDSAT Data File

In [ ]:
fname_ds = create_xarray_dataset_from_file(fname)

In [ ]:
fname_ds

## <font color="red">Plotting Our Data</font>

In [ ]:
fname_GB = fname_ds.nbytes / (1024*1024*1024)

fname_GB

In [ ]:
toa_band6 = fname_ds['toa_band6']

A basic image plot:

In [ ]:
toa_band6.plot()

Use Cartopy for a contour plot:

In [ ]:
lon_m = toa_band6.lon.values.mean()
lat_m = toa_band6.lat.values.mean()

map_projection = ccrs.LambertAzimuthalEqualArea(
                      central_longitude=lon_m, 
                      central_latitude=lat_m
                )
data_transform = ccrs.PlateCarree()

subplot_kw = dict(projection=map_projection)
fig, ax = plt.subplots(1, 1,
                       figsize=(15, 9),
                       subplot_kw=subplot_kw)

units = toa_band6.attrs['units']
cbar_kwargs = {'orientation':'horizontal', 
               'shrink':0.6, "pad" : .05, 
               'aspect':40, 'label': units}

toa_band6.plot.pcolormesh(ax=ax, x='lon', y='lat',
                          transform=data_transform,
                          cbar_kwargs=cbar_kwargs,
                          add_colorbar=True,
                          cmap="jet"
                         )

ax.coastlines()
plt.title(field_name, fontsize=8)
plt.tight_layout();

A basic `hvPlot` plot.

In [ ]:
toa_band6.hvplot.image().opts(cmap = 'jet', height=650, width=1300)   # Doesn't work

An intermediate `hvPlot` plot.

In [ ]:
toa_band6.hvplot.quadmesh('lon', 'lat', cmap = 'jet')

In [ ]:
toa_band6.hvplot.quadmesh('lon', 'lat', 
                          xlim = (-172.5, -166.75), 
                          ylim = (63, 65.3), 
                          geo = True, project = True, 
                          rasterize = True, 
                          projection = ccrs.PlateCarree(), 
                          features = ['borders'], coastline = True)

## Exercise

Use the file:

```
new_fname = os.path.join(data_dir, "LT50830152011214GLC00.hdf")
```

To create an Xarray Dataset and plot any field of your choice.

<details><summary><b><font color="blue">Click here for the solution to Exercise</font></b></summary>
<p>

```python
new_fname = os.path.join(data_dir, "LT50830152011214GLC00.hdf")

new_fname_ds = create_xarray_dataset_from_file(new_fname)

new_fname_ds['toa_band6'].plot()

```
    
</p>



</details>